In [1]:
from pymodm.connection import connect

name = 'admin'
password = 'admin123456'
ip = '127.0.0.1'
port = 27017

def connect_db(name, password, ip, port, db_name):
    uri = "mongodb://{}:{}@{}:{}/{}?authSource=admin".format(name, password, ip, port, db_name)
    connect(uri, db_name)

In [ ]:
from pymodm import fields, MongoModel
db_alias = 'cfg'
collection_name = 'cc_config'
class ExchangeConfiguration(MongoModel):
    """Exchange configuration"""
    identifier = fields.IntegerField(primary_key=True)
    app_key = fields.CharField(min_length=3)
    app_secret = fields.CharField(min_length=3)
    name = fields.CharField(min_length=3)
    deleted = fields.BooleanField(default=False)

    class Meta:
        collection_name = collection_name
        connection_alias = db_alias
